# Preparing Raw Data (Preparando conjunto de dados)

This is necessary to try any subsection of this Notebook, so execute it!

---

Essa parte é necessária para executar qualquer subseção desse Notebook, portanto, é necessário executá-la sempre.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Literal, Any

In [ ]:
TESTED_MODES = np.arange(1, 33, dtype=int)
TESTED_MODES_FULL_ORDER = np.arange(1, 100, 5, dtype=int)
TRAIN_SPLIT = 2/3
# Kernels disponíveis para uso pelo SKLearn
USED_KERNEL: Literal['linear', 'poly', 'rbf', 'sigmoid'] = 'rbf'
kernel_name = 'radial basis function'

In [ ]:
np.random.seed(42)

In [ ]:
collision_file = np.load(".data/Time_series_colisao.npz")
display(collision_file)
spreading_file = np.load(".data/Time_series_espalhamento.npz")
display(spreading_file)

In [ ]:
collision_timesteps_raw = collision_file['TS']
display(f"Dimensions of the collision simulations: {collision_timesteps_raw.shape}")
display(f"That is, {collision_timesteps_raw.shape[0]} simulations, with {collision_timesteps_raw.shape[1]} timesteps each, having {collision_timesteps_raw.shape[2]} components each.")
spreading_timesteps_raw = spreading_file['TS']
display(f"Dimensions of the spreading simulations: {spreading_timesteps_raw.shape}")
display(f"That is, {spreading_timesteps_raw.shape[0]} simulations, with {spreading_timesteps_raw.shape[1]} timesteps each, having {spreading_timesteps_raw.shape[2]} components each.")

# Collision component-wise order reduction

## Reordering data (reordenação de dados)

We want to avoid mixing components in this case

---

Precisamos separar as componentes nesse caso

In [ ]:
components_names = ['Diameter', 'Height', 'Kinetic Energy', 'Surface Energy', 'Dissipative Energy']
components_names_pt = ['Diâmetro', 'Altura', 'Energia Cinética', 'Energia Superficial', 'Energia Dissipativa']
components_names_pt_treated = ['diametro', 'altura', 'energia_cinetica', 'energia_superficial', 'energia_dissipativa']
collision: dict[str, Any] = dict()
for i, cn in enumerate(components_names):
    cn_dict = {}
    cn_dict['original_data'] = collision_timesteps_raw[:, :, i].T
    cn_dict['pt_name'] = components_names_pt[i]
    cn_dict['pt_name_treated'] = components_names_pt_treated[i]
    # (n_simulations, timesteps) [RAW]
    # (timesteps, n_simulations) [.T]
    collision[cn] = cn_dict

## Scaling

Firstly, we want to remove the "temporal mean", that is, the mean for each simulation of each component. After that, we can also scale the data so its standard variation is equal to 1:

$$\pmb{\hat{X}}=\sqrt{\frac{1}{n}}\left(\pmb{X}-\hat{x}1\right)$$

---

Aplica-se uma estandardização para que a média e desvio padrão de cada coluna seja igual a 0 e 1, respectivamente.

In [ ]:
from sklearn.preprocessing import StandardScaler

for key, obj in collision.items():
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(obj['original_data'])
    display(f"Asserting mean = 0 and std = 1 for component '{key}'")
    assert np.allclose(scaled_data.mean(axis=0), 0)
    assert np.allclose(scaled_data.std(axis=0), 1)
    display("Success!")
    obj['scaled_data'] = scaled_data
    obj['scaler'] = scaler

## Testing for all the given modes

In [ ]:
from sklearn.decomposition import KernelPCA

NUMBER_OF_MODES = TESTED_MODES.shape[0]
for key, obj in collision.items():
    number_of_timesteps = obj['scaled_data'].shape[0]
    permutation_idx = np.random.permutation(number_of_timesteps)
    training_size = int(TRAIN_SPLIT * number_of_timesteps)
    training_idx = permutation_idx[:training_size]
    testing_idx = permutation_idx[training_size:]
    training_data = obj['original_data'][training_idx]
    training_data_scaled = obj['scaled_data'][training_idx]
    testing_data = obj['original_data'][testing_idx]
    testing_data_scaled = obj['scaled_data'][testing_idx]
    modes_mse = np.zeros((number_of_timesteps, NUMBER_OF_MODES))
    modes_mae = np.zeros_like(modes_mse)
    modes_mrae = np.zeros_like(modes_mse)
    for i, mode in enumerate(TESTED_MODES):
        # TRAINING DATA
        kpca = KernelPCA(n_components=mode, random_state=42, kernel=USED_KERNEL, fit_inverse_transform=True)
        kpca.fit(training_data_scaled)
        reconstructed_training_scaled = kpca.inverse_transform(kpca.transform(training_data_scaled))
        reconstructed_training = obj['scaler'].inverse_transform(reconstructed_training_scaled)
        mse_training = np.mean((training_data - reconstructed_training)**2, axis=1)
        mae_training = np.mean(np.abs(training_data - reconstructed_training), axis=1)
        mrae_training = np.mean(np.abs(training_data - reconstructed_training) / np.mean(training_data), axis=1)
        # TESTING DATA
        reconstructed_testing_scaled = kpca.inverse_transform(kpca.transform(testing_data_scaled))
        reconstructed_testing = obj['scaler'].inverse_transform(reconstructed_testing_scaled)
        mse_testing = np.mean((testing_data - reconstructed_testing)**2, axis=1)
        mae_testing = np.mean(np.abs(testing_data - reconstructed_testing), axis=1)
        mrae_testing = np.mean(np.abs(testing_data - reconstructed_testing) / np.abs(testing_data), axis=1)
        # STORING DATA
        modes_mse[training_idx, i] = mse_training
        modes_mae[training_idx, i] = mae_training
        modes_mrae[training_idx, i] = mrae_training
        modes_mse[testing_idx, i] = mse_testing
        modes_mae[testing_idx, i] = mae_testing
        modes_mrae[testing_idx, i] = mrae_testing
    training_error_idx = pd.Index(training_idx, name='Timestep')
    testing_error_idx = pd.Index(testing_idx, name='Timestep')
    columns = pd.MultiIndex.from_product([TESTED_MODES, ['MSE', 'MAE', 'MRAE']], names=['Modes', 'Metric'])
    training_error_data = np.zeros((training_error_idx.shape[0], NUMBER_OF_MODES * 3))
    testing_error_data = np.zeros((testing_error_idx.shape[0], NUMBER_OF_MODES * 3))
    training_error_data[:, ::3] = modes_mse[training_idx]
    training_error_data[:, 1::3] = modes_mae[training_idx]
    training_error_data[:, 2::3] = modes_mrae[training_idx]
    testing_error_data[:, ::3] = modes_mse[testing_idx]
    testing_error_data[:, 1::3] = modes_mae[testing_idx]
    testing_error_data[:, 2::3] = modes_mrae[testing_idx]
    training_error_df = pd.DataFrame(training_error_data, columns=columns, index=training_error_idx).sort_index()
    testing_error_df = pd.DataFrame(testing_error_data, columns=columns, index=testing_error_idx).sort_index()
    obj['permutation_idx'] = permutation_idx
    obj['training_idx'] = training_idx
    obj['testing_idx'] = testing_idx
    obj['training_error'] = training_error_df
    obj['testing_error'] = testing_error_df

## Showing these errors in graphs

In [ ]:
BASE_PATH = f".results/kpca/{USED_KERNEL}/"

In [ ]:
import os
from matplotlib.ticker import MaxNLocator, PercentFormatter

fig, ax = plt.subplots(figsize=(16, 9))
os.makedirs(BASE_PATH, exist_ok=True)

for key, obj in collision.items():
    training_error = obj['training_error'].loc[:,
                                               (slice(None), 'MRAE')].mean().unstack() * 100
    testing_error = obj['testing_error'].loc[:,
                                             (slice(None), 'MRAE')].mean().unstack() * 100
    # region COMPONENT-SPECIFIC FIGURE
    component_fig, component_ax = plt.subplots(1, 2, figsize=(16, 9))
    training_error.plot(ax=component_ax[0], linestyle='--', alpha=0.5)
    testing_error.plot(ax=component_ax[0], linestyle='-')
    component_ax[0].set_ylabel("Erro médio absoluto percentual (MAPE)")
    component_ax[0].set_xlabel("Número de modos")
    component_ax[1].remove()
    component_ax[1] = component_fig.add_subplot(1, 2, 2, projection='3d')
    x, y = np.meshgrid(obj['testing_error'].index, TESTED_MODES)
    z = obj['testing_error'].loc[:, (slice(None), 'MRAE')].values.T * 100
    component_ax[1].plot_surface(x, y, z)
    component_ax[1].set_xlabel("Snapshot")
    component_ax[1].set_ylabel("Número de modos")
    component_ax[1].set_zlabel("Erro médio absoluto percentual (MAPE)")
    component_ax[1].set_title("Erro de teste")
    max_error_last_mode_train = obj["training_error"].loc[:,
                                                          (TESTED_MODES[-1], 'MRAE')].max() * 100
    max_error_last_mode_test = obj["testing_error"].loc[:,
                                                        (TESTED_MODES[-1], 'MRAE')].max() * 100
    max_error_last_modes = max(
        max_error_last_mode_train, max_error_last_mode_test)
    component_ax[0].scatter(
        [TESTED_MODES[-1]], [max_error_last_modes], color='red')
    component_ax[0].annotate(f"{max_error_last_modes:.2f}%", (TESTED_MODES[-1],
                             max_error_last_modes), textcoords="offset points", xytext=(0, 10), ha='center')
    component_ax[0].legend(
        ['Treinamento', 'Teste', 'Erro máximo no melhor caso'])
    component_ax[0].yaxis.set_major_locator(MaxNLocator())
    component_ax[0].yaxis.set_major_formatter(PercentFormatter())
    component_ax[1].zaxis.set_major_locator(MaxNLocator())
    component_ax[1].zaxis.set_major_formatter(PercentFormatter())
    component_fig.suptitle(
        f"Erro médio absoluto percentual para a componente '{obj["pt_name"]}'")
    os.makedirs(f"{BASE_PATH}/collision/separate", exist_ok=True)
    component_fig.suptitle(f"""Simulação da colisão
                           Kernel PCA com kernel {kernel_name}: Redução da componente '{obj["pt_name"]}'""")
    component_fig.tight_layout()
    component_fig.savefig(
        f"{BASE_PATH}/collision/separate/{obj["pt_name_treated"]}.png", dpi=333, bbox_inches='tight')
    # endregion
    training_error.plot(ax=ax, linestyle='--', alpha=0.5)
    testing_error.plot(ax=ax, linestyle='-')

training_legend = [
    f"{cn["pt_name"]} (treinamento)" for cn in collision.values()]
testing_legend = [f"{cn["pt_name"]} (teste)" for cn in collision.values()]
legend = [''] * (len(training_legend) + len(testing_legend))
legend[::2] = training_legend
legend[1::2] = testing_legend
ax.legend(legend)
ax.yaxis.set_major_locator(MaxNLocator())
ax.yaxis.set_major_formatter(PercentFormatter())
ax.set_title(f"""Simulação da colisão
             Kernel PCA com kernel {kernel_name}: Componentes separadas""")
ax.set_xlabel("Número de modos")
ax.set_ylabel("Erro médio absoluto percentual (MAPE)")
fig.tight_layout()
fig.savefig(f"{BASE_PATH}/collision/separate/todos.png", dpi=333, bbox_inches='tight')
plt.show()

# Spreading component-wise order reduction

## Reordering data (reordenação de dados)

We want to avoid mixing components in this case

---

Precisamos separar as componentes nesse caso

In [ ]:
spreading_components_names = ['Diameter', 'Kinetic Energy', 'Surface Energy', 'Dissipative Energy']
spreading_components_names_pt = ['Diâmetro', 'Energia Cinética', 'Energia Superficial', 'Energia Dissipativa']
spreading_components_names_pt_treated = ['diametro', 'energia_cinetica', 'energia_superficial', 'energia_dissipativa']
spreading = dict()
for i, cn in enumerate(spreading_components_names):
    cn_dict = {}
    cn_dict['original_data'] = spreading_timesteps_raw[:, :, i].T
    cn_dict['pt_name'] = spreading_components_names_pt[i]
    cn_dict['pt_name_treated'] = spreading_components_names_pt_treated[i]
    # (n_simulations, timesteps) [RAW]
    # (timesteps, n_simulations) [.T]
    spreading[cn] = cn_dict

## Scaling

Firstly, we want to remove the "temporal mean", that is, the mean for each simulation of each component. After that, we can also scale the data so its standard variation is equal to 1:

$$\pmb{\hat{X}}=\sqrt{\frac{1}{n}}\left(\pmb{X}-\hat{x}1\right)$$

---

Aplica-se uma estandardização para que a média e desvio padrão de cada coluna seja igual a 0 e 1, respectivamente.

In [ ]:
from sklearn.preprocessing import StandardScaler

for key, obj in spreading.items():
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(obj['original_data'])
    display(f"Asserting mean = 0 and std = 1 for component '{key}'")
    assert np.allclose(scaled_data.mean(axis=0), 0)
    assert np.allclose(scaled_data.std(axis=0), 1)
    display("Success!")
    obj['scaled_data'] = scaled_data
    obj['scaler'] = scaler

## Testing for all the given modes

In [ ]:
from sklearn.decomposition import KernelPCA

NUMBER_OF_MODES = TESTED_MODES.shape[0]
for key, obj in spreading.items():
    number_of_timesteps = obj['scaled_data'].shape[0]
    permutation_idx = np.random.permutation(number_of_timesteps)
    training_size = int(TRAIN_SPLIT * number_of_timesteps)
    training_idx = permutation_idx[:training_size]
    testing_idx = permutation_idx[training_size:]
    training_data = obj['original_data'][training_idx]
    training_data_scaled = obj['scaled_data'][training_idx]
    testing_data = obj['original_data'][testing_idx]
    testing_data_scaled = obj['scaled_data'][testing_idx]
    modes_mse = np.zeros((number_of_timesteps, NUMBER_OF_MODES))
    modes_mae = np.zeros_like(modes_mse)
    modes_mrae = np.zeros_like(modes_mse)
    for i, mode in enumerate(TESTED_MODES):
        # TRAINING DATA
        kpca = KernelPCA(n_components=mode, random_state=42, kernel=USED_KERNEL, fit_inverse_transform=True)
        kpca.fit(training_data_scaled)
        reconstructed_training_scaled = kpca.inverse_transform(kpca.transform(training_data_scaled))
        reconstructed_training = obj['scaler'].inverse_transform(reconstructed_training_scaled)
        mse_training = np.mean((training_data - reconstructed_training)**2, axis=1)
        mae_training = np.mean(np.abs(training_data - reconstructed_training), axis=1)
        mrae_training = np.mean(np.abs(training_data - reconstructed_training) / np.abs(training_data), axis=1)
        # TESTING DATA
        reconstructed_testing_scaled = kpca.inverse_transform(kpca.transform(testing_data_scaled))
        reconstructed_testing = obj['scaler'].inverse_transform(reconstructed_testing_scaled)
        mse_testing = np.mean((testing_data - reconstructed_testing)**2, axis=1)
        mae_testing = np.mean(np.abs(testing_data - reconstructed_testing), axis=1)
        mrae_testing = np.mean(np.abs(testing_data - reconstructed_testing) / np.abs(testing_data), axis=1)
        # STORING DATA
        modes_mse[training_idx, i] = mse_training
        modes_mae[training_idx, i] = mae_training
        modes_mrae[training_idx, i] = mrae_training
        modes_mse[testing_idx, i] = mse_testing
        modes_mae[testing_idx, i] = mae_testing
        modes_mrae[testing_idx, i] = mrae_testing
    training_error_idx = pd.Index(training_idx, name='Timestep')
    testing_error_idx = pd.Index(testing_idx, name='Timestep')
    columns = pd.MultiIndex.from_product([TESTED_MODES, ['MSE', 'MAE', 'MRAE']], names=['Modes', 'Metric'])
    training_error_data = np.zeros((training_error_idx.shape[0], NUMBER_OF_MODES * 3))
    testing_error_data = np.zeros((testing_error_idx.shape[0], NUMBER_OF_MODES * 3))
    training_error_data[:, ::3] = modes_mse[training_idx]
    training_error_data[:, 1::3] = modes_mae[training_idx]
    training_error_data[:, 2::3] = modes_mrae[training_idx]
    testing_error_data[:, ::3] = modes_mse[testing_idx]
    testing_error_data[:, 1::3] = modes_mae[testing_idx]
    testing_error_data[:, 2::3] = modes_mrae[testing_idx]
    training_error_df = pd.DataFrame(training_error_data, columns=columns, index=training_error_idx).sort_index()
    testing_error_df = pd.DataFrame(testing_error_data, columns=columns, index=testing_error_idx).sort_index()
    obj['permutation_idx'] = permutation_idx
    obj['training_idx'] = training_idx
    obj['testing_idx'] = testing_idx
    obj['training_error'] = training_error_df
    obj['testing_error'] = testing_error_df

## Showing these errors in graphs

In [ ]:
import os
from matplotlib.ticker import MaxNLocator, PercentFormatter

fig, ax = plt.subplots(figsize=(16, 9))

for key, obj in spreading.items():
    training_error = obj['training_error'].loc[:,
                                               (slice(None), 'MRAE')].mean().unstack() * 100
    testing_error = obj['testing_error'].loc[:,
                                             (slice(None), 'MRAE')].mean().unstack() * 100
    # region COMPONENT-SPECIFIC FIGURE
    component_fig, component_ax = plt.subplots(1, 2, figsize=(16, 9))
    training_error.plot(ax=component_ax[0], linestyle='--', alpha=0.5)
    testing_error.plot(ax=component_ax[0], linestyle='-')
    component_ax[0].set_ylabel("Erro médio absoluto percentual (MAPE)")
    component_ax[0].set_xlabel("Número de modos")
    component_ax[1].remove()
    component_ax[1] = component_fig.add_subplot(1, 2, 2, projection='3d')
    x, y = np.meshgrid(obj['testing_error'].index, TESTED_MODES)
    z = obj['testing_error'].loc[:, (slice(None), 'MRAE')].values.T * 100
    component_ax[1].plot_surface(x, y, z)
    component_ax[1].set_xlabel("Snapshot")
    component_ax[1].set_ylabel("Número de modos")
    component_ax[1].set_zlabel("Erro médio absoluto percentual (MAPE)")
    component_ax[1].set_title("Erro de teste")
    max_error_last_mode_train = obj["training_error"].loc[:,
                                                          (TESTED_MODES[-1], 'MRAE')].max() * 100
    max_error_last_mode_test = obj["testing_error"].loc[:,
                                                        (TESTED_MODES[-1], 'MRAE')].max() * 100
    max_error_last_modes = max(
        max_error_last_mode_train, max_error_last_mode_test)
    component_ax[0].scatter(
        [TESTED_MODES[-1]], [max_error_last_modes], color='red')
    component_ax[0].annotate(f"{max_error_last_modes:.2f}%", (TESTED_MODES[-1],
                             max_error_last_modes), textcoords="offset points", xytext=(0, 10), ha='center')
    component_ax[0].legend(
        ['Treinamento', 'Teste', 'Erro máximo no melhor caso'])
    component_ax[0].yaxis.set_major_locator(MaxNLocator())
    component_ax[0].yaxis.set_major_formatter(PercentFormatter())
    component_ax[1].zaxis.set_major_locator(MaxNLocator())
    component_ax[1].zaxis.set_major_formatter(PercentFormatter())
    component_fig.suptitle(
        f"Erro médio absoluto percentual para a componente '{obj["pt_name"]}'")
    component_fig.tight_layout()
    os.makedirs(f"{BASE_PATH}/spreading/separate", exist_ok=True)
    component_fig.suptitle(f"""Simulação do espalhamento
                           Kernel PCA com kernel {kernel_name}: Redução da componente '{obj["pt_name"]}'""")
    component_fig.subplots_adjust(top=0.9)
    component_fig.savefig(
        f"{BASE_PATH}/spreading/separate/{obj["pt_name_treated"]}.png", dpi=333, bbox_inches='tight')
    # endregion
    training_error.plot(ax=ax, linestyle='--', alpha=0.5)
    testing_error.plot(ax=ax, linestyle='-')

training_legend = [
    f"{cn["pt_name"]} (treinamento)" for cn in collision.values()]
testing_legend = [f"{cn["pt_name"]} (teste)" for cn in collision.values()]
legend = [''] * (len(training_legend) + len(testing_legend))
legend[::2] = training_legend
legend[1::2] = testing_legend
ax.legend(legend)
ax.yaxis.set_major_locator(MaxNLocator())
ax.yaxis.set_major_formatter(PercentFormatter())
ax.set_title(f"""Simulação de espalhamento
            Kernel PCA com kernel {kernel_name}: Componentes separadas""")
ax.set_xlabel("Número de modos")
ax.set_ylabel("Erro médio absoluto percentual (MAPE)")
fig.tight_layout()
fig.savefig(f"{BASE_PATH}/spreading/separate/todos.png", dpi=333)
plt.show()

# Collision full order reduction

"Full order reduction" is the case when there are no separate components. In this case, every component is present in the dimensionality reduction.

---

Redução de dimensionalidade "full order" é o nome que iremos dar a reduções de dimensionalidade em que todas as componentes estão presentes, isto é, sem separação de componentes.

## Reordering data and rescaling energies

In [ ]:
collision_timesteps_components_last = np.transpose(collision_timesteps_raw, (1, 0, 2))
display(f"Shape after transpose is: {collision_timesteps_components_last.shape}")
# Flattening 2nd and 3rd dimensions
collision_timesteps_components_last = (
    collision_timesteps_components_last
    .reshape(collision_timesteps_components_last.shape[0], -1)
)
display(f"Shape after flattening is: {collision_timesteps_components_last.shape}")
collision_full = {}
collision_full['original_data'] = collision_timesteps_components_last

## Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(collision_full['original_data'])
display(f"Asserting mean = 0 and std = 1 for every column")
assert np.allclose(scaled_data.mean(axis=0), 0)
assert np.allclose(scaled_data.std(axis=0), 1)
display("Success!")
collision_full['scaled_data'] = scaled_data
collision_full['scaler'] = scaler

## Testing every mode

In [ ]:
from sklearn.decomposition import KernelPCA

NUMBER_OF_MODES = TESTED_MODES_FULL_ORDER.shape[0]
number_of_timesteps = collision_full['scaled_data'].shape[0]
permutation_idx = np.random.permutation(number_of_timesteps)
training_size = int(TRAIN_SPLIT * number_of_timesteps)
training_idx = permutation_idx[:training_size]
testing_idx = permutation_idx[training_size:]
training_data = collision_full['original_data'][training_idx]
training_data_scaled = collision_full['scaled_data'][training_idx]
testing_data = collision_full['original_data'][testing_idx]
testing_data_scaled = collision_full['scaled_data'][testing_idx]
modes_mse = np.zeros((number_of_timesteps, NUMBER_OF_MODES))
modes_mae = np.zeros_like(modes_mse)
modes_mrae = np.zeros_like(modes_mse)
for i, mode in enumerate(TESTED_MODES_FULL_ORDER):
    # TRAINING DATA
    kpca = KernelPCA(n_components=mode, random_state=42, kernel=USED_KERNEL, fit_inverse_transform=True)
    kpca.fit(training_data_scaled)
    reconstructed_training_scaled = kpca.inverse_transform(kpca.transform(training_data_scaled))
    reconstructed_training = collision_full['scaler'].inverse_transform(reconstructed_training_scaled)
    mse_training = np.mean((training_data - reconstructed_training)**2, axis=1)
    mae_training = np.mean(np.abs(training_data - reconstructed_training), axis=1)
    mrae_training = np.mean(np.abs(training_data - reconstructed_training) / np.abs(training_data), axis=1)
    # TESTING DATA
    reconstructed_testing_scaled = kpca.inverse_transform(kpca.transform(testing_data_scaled))
    reconstructed_testing = collision_full['scaler'].inverse_transform(reconstructed_testing_scaled)
    mse_testing = np.mean((testing_data - reconstructed_testing)**2, axis=1)
    mae_testing = np.mean(np.abs(testing_data - reconstructed_testing), axis=1)
    mrae_testing = np.mean(np.abs(testing_data - reconstructed_testing) / np.abs(testing_data), axis=1)
    # STORING DATA
    modes_mse[training_idx, i] = mse_training
    modes_mae[training_idx, i] = mae_training
    modes_mrae[training_idx, i] = mrae_training
    modes_mse[testing_idx, i] = mse_testing
    modes_mae[testing_idx, i] = mae_testing
    modes_mrae[testing_idx, i] = mrae_testing
training_error_idx = pd.Index(training_idx, name='Timestep')
testing_error_idx = pd.Index(testing_idx, name='Timestep')
columns = pd.MultiIndex.from_product([TESTED_MODES_FULL_ORDER, ['MSE', 'MAE', 'MRAE']], names=['Modes', 'Metric'])
training_error_data = np.zeros((training_error_idx.shape[0], NUMBER_OF_MODES * 3))
testing_error_data = np.zeros((testing_error_idx.shape[0], NUMBER_OF_MODES * 3))
training_error_data[:, ::3] = modes_mse[training_idx]
training_error_data[:, 1::3] = modes_mae[training_idx]
training_error_data[:, 2::3] = modes_mrae[training_idx]
testing_error_data[:, ::3] = modes_mse[testing_idx]
testing_error_data[:, 1::3] = modes_mae[testing_idx]
testing_error_data[:, 2::3] = modes_mrae[testing_idx]
training_error_df = pd.DataFrame(training_error_data, columns=columns, index=training_error_idx).sort_index()
testing_error_df = pd.DataFrame(testing_error_data, columns=columns, index=testing_error_idx).sort_index()
collision_full['permutation_idx'] = permutation_idx
collision_full['training_idx'] = training_idx
collision_full['testing_idx'] = testing_idx
collision_full['training_error'] = training_error_df
collision_full['testing_error'] = testing_error_df

## Graphing

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 9))
#region LINE GRAPH
(
    (
        collision_full['training_error']
        .loc[:, (slice(None), 'MRAE')]
        .mean()
        .unstack()
        * 100
    )
    .plot(ax=ax[0], linestyle='--', alpha=0.5)
)
(
    (
        collision_full['testing_error']
        .loc[:, (slice(None), 'MRAE')]
        .mean()
        .unstack()
        * 100
    )
    .plot(ax=ax[0], linestyle='-')
)

ax[0].legend([
    'Training',
    'Testing',
])
ax[0].set_xlabel("Número de modos")
ax[0].set_ylabel("Erro médio absoluto percentual (MAPE)")
#endregion

#region 3D SURFACE GRAPH
ax[1].remove()
ax[1] = fig.add_subplot(1, 2, 2, projection='3d')
x, y = np.meshgrid(collision_full['testing_error'].index, TESTED_MODES_FULL_ORDER)
z = collision_full['testing_error'].loc[:, (slice(None), 'MRAE')].values.T * 100
ax[1].plot_surface(x, y, z)
ax[1].set_xlabel("Snapshot")
ax[1].set_ylabel("Número de modos")
ax[1].set_zlabel("Erro médio absoluto percentual (MAPE)")
ax[1].set_title("Erro de teste")
#endregion

#region AXES FORMATTTING
ax[0].yaxis.set_major_locator(MaxNLocator())
ax[0].yaxis.set_major_formatter(PercentFormatter())
ax[1].zaxis.set_major_locator(MaxNLocator())
ax[1].zaxis.set_major_formatter(PercentFormatter())
fig.suptitle(
    f"""Kernel PCA com kernel {kernel_name}: Simulação da colisão
    Erro médio absoluto percentual para redução sem separação de componentes""")
fig.tight_layout()
#endregion

#region SAVING

os.makedirs(f"{BASE_PATH}/collision", exist_ok=True)
fig.savefig(f"{BASE_PATH}/collision/full_order.png", dpi=333)
#endregion

plt.show()

# Spreading full order reduction

## Reordering data and rescaling energies

In [ ]:
spreading_timesteps_components_last = np.transpose(spreading_timesteps_raw, (1, 0, 2))
display(f"Shape after transpose is: {spreading_timesteps_components_last.shape}")
# Flattening 2nd and 3rd dimensions
spreading_timesteps_components_last = (
    spreading_timesteps_components_last
    .reshape(spreading_timesteps_components_last.shape[0], -1)
)
display(f"Shape after flattening is: {spreading_timesteps_components_last.shape}")
spreading_full = {}
spreading_full['original_data'] = spreading_timesteps_components_last

## Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(spreading_full['original_data'])
display(f"Asserting mean = 0 and std = 1 for every column")
assert np.allclose(scaled_data.mean(axis=0), 0)
assert np.allclose(scaled_data.std(axis=0), 1)
display("Success!")
spreading_full['scaled_data'] = scaled_data
spreading_full['scaler'] = scaler

## Testing every mode

In [ ]:
from sklearn.decomposition import KernelPCA

NUMBER_OF_MODES = TESTED_MODES_FULL_ORDER.shape[0]
number_of_timesteps = spreading_full['scaled_data'].shape[0]
permutation_idx = np.random.permutation(number_of_timesteps)
training_size = int(TRAIN_SPLIT * number_of_timesteps)
training_idx = permutation_idx[:training_size]
testing_idx = permutation_idx[training_size:]
training_data = spreading_full['original_data'][training_idx]
training_data_scaled = spreading_full['scaled_data'][training_idx]
testing_data = spreading_full['original_data'][testing_idx]
testing_data_scaled = spreading_full['scaled_data'][testing_idx]
modes_mse = np.zeros((number_of_timesteps, NUMBER_OF_MODES))
modes_mae = np.zeros_like(modes_mse)
modes_mrae = np.zeros_like(modes_mse)
for i, mode in enumerate(TESTED_MODES_FULL_ORDER):
    # TRAINING DATA
    kpca = KernelPCA(n_components=mode, random_state=42, kernel=USED_KERNEL, fit_inverse_transform=True)
    kpca.fit(training_data_scaled)
    reconstructed_training_scaled = kpca.inverse_transform(kpca.transform(training_data_scaled))
    reconstructed_training = spreading_full['scaler'].inverse_transform(reconstructed_training_scaled)
    mse_training = np.mean((training_data - reconstructed_training)**2, axis=1)
    mae_training = np.mean(np.abs(training_data - reconstructed_training), axis=1)
    mrae_training = np.mean(np.abs(training_data - reconstructed_training) / np.abs(training_data), axis=1)
    # TESTING DATA
    reconstructed_testing_scaled = kpca.inverse_transform(kpca.transform(testing_data_scaled))
    reconstructed_testing = spreading_full['scaler'].inverse_transform(reconstructed_testing_scaled)
    mse_testing = np.mean((testing_data - reconstructed_testing)**2, axis=1)
    mae_testing = np.mean(np.abs(testing_data - reconstructed_testing), axis=1)
    mrae_testing = np.mean(np.abs(testing_data - reconstructed_testing) / np.abs(testing_data), axis=1)
    # STORING DATA
    modes_mse[training_idx, i] = mse_training
    modes_mae[training_idx, i] = mae_training
    modes_mrae[training_idx, i] = mrae_training
    modes_mse[testing_idx, i] = mse_testing
    modes_mae[testing_idx, i] = mae_testing
    modes_mrae[testing_idx, i] = mrae_testing
training_error_idx = pd.Index(training_idx, name='Timestep')
testing_error_idx = pd.Index(testing_idx, name='Timestep')
columns = pd.MultiIndex.from_product([TESTED_MODES_FULL_ORDER, ['MSE', 'MAE', 'MRAE']], names=['Modes', 'Metric'])
training_error_data = np.zeros((training_error_idx.shape[0], NUMBER_OF_MODES * 3))
testing_error_data = np.zeros((testing_error_idx.shape[0], NUMBER_OF_MODES * 3))
training_error_data[:, ::3] = modes_mse[training_idx]
training_error_data[:, 1::3] = modes_mae[training_idx]
training_error_data[:, 2::3] = modes_mrae[training_idx]
testing_error_data[:, ::3] = modes_mse[testing_idx]
testing_error_data[:, 1::3] = modes_mae[testing_idx]
testing_error_data[:, 2::3] = modes_mrae[testing_idx]
training_error_df = pd.DataFrame(training_error_data, columns=columns, index=training_error_idx).sort_index()
testing_error_df = pd.DataFrame(testing_error_data, columns=columns, index=testing_error_idx).sort_index()
spreading_full['permutation_idx'] = permutation_idx
spreading_full['training_idx'] = training_idx
spreading_full['testing_idx'] = testing_idx
spreading_full['training_error'] = training_error_df
spreading_full['testing_error'] = testing_error_df

## Graphing

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 9))
# region LINE GRAPH
(
    (
        spreading_full['training_error']
        .loc[:, (slice(None), 'MRAE')]
        .mean()
        .unstack()
        * 100
    )
    .plot(ax=ax[0], linestyle='--', alpha=0.5)
)
(
    (
        spreading_full['testing_error']
        .loc[:, (slice(None), 'MRAE')]
        .mean()
        .unstack()
        * 100
    )
    .plot(ax=ax[0], linestyle='-')
)

ax[0].legend([
    'Training',
    'Testing',
])
ax[0].set_xlabel("Número de modos")
ax[0].set_ylabel("Erro médio absoluto percentual (MAPE)")
# endregion

# region 3D SURFACE GRAPH
ax[1].remove()
ax[1] = fig.add_subplot(1, 2, 2, projection='3d')
x, y = np.meshgrid(
    spreading_full['testing_error'].index, TESTED_MODES_FULL_ORDER)
z = spreading_full['testing_error'].loc[:,
                                        (slice(None), 'MRAE')].values.T * 100
ax[1].plot_surface(x, y, z)
ax[1].set_xlabel("Snapshot")
ax[1].set_ylabel("Número de modos")
ax[1].set_zlabel("Erro médio absoluto percentual (MAPE)")
ax[1].set_title("Erro de teste")
# endregion

# region AXES FORMATTTING
ax[0].yaxis.set_major_locator(MaxNLocator())
ax[0].yaxis.set_major_formatter(PercentFormatter())
ax[1].zaxis.set_major_locator(MaxNLocator())
ax[1].zaxis.set_major_formatter(PercentFormatter())
fig.suptitle(
    f"""Kernel PCA com kernel {kernel_name}: Simulação do espalhamento
    Erro médio absoluto percentual para redução sem separação de componentes""")
fig.tight_layout()
# endregion

# region SAVING

os.makedirs(f"{BASE_PATH}/spreading", exist_ok=True)
fig.savefig(f"{BASE_PATH}/spreading/full_order.png", dpi=333)
# endregion

plt.show()